# Introduction to LLM Agents with LangChain

Welcome to the workshop on building LLM agents with LangChain!
    
**The use case:** 

Winter holidays are coming up and you still don't know where to go. Oh no! 

You decide to build a program that helps you get information on holiday locations. For example, you would like to to find out how big a specific city is, what sights are there to see, how the weather there is, and you would like to get a drawing of that place, to get a first impression. Because who does not like art? 

You will implement this through an LLM agent, who has access to
* the wikipedia API,
* a weather API,
* can generate images by using a HuggingFace API. 

---
## ⚙️ Setup

Before we start, let's set up the workspace.

### APIs


#### Weather API 
We will use the Weather API from `visualcrossing`. You have to generate you API key that you can later use to access this API. Follow the steps below: 

1. Signup up at https://www.visualcrossing.com/
2. Verify your account
3. Sign in and click on `Account` (blue button in the top right corner)
4. Under `Details` you should be able to see a `Key`
5. Copy the Key in `helper_functions/keys.py`


#### HuggingFace Token
We will use a model avaialble through a HuggingFace API. For that you need to generate a Token. Follow the steps below: 

1. Visit [HuggingFace](https://huggingface.co/) and sign up or log in.
2. Go to your profile (click your avatar), then "Settings" > "Access Tokens."
3. Click "New Token," select `Fine-grained` as Token Type role, and check the box `Make calls to the serverless Inference API`
4. Copy the Token in `helper_functions/keys.py`.


### Repository
To set up Google Colab, follow the steps below. If you would rather run the notebooks locally, visit the `README`

1. Open [Google colab](https://colab.research.google.com/) > Open notebook > GitHub 
2. Paste the [repository link](https://github.com/mkmbader/pydata_workshop_September2024) 
3. Click on “1_workshop_agent_with_langchain.ipynb”


In [ ]:
# RUN THIS IF YOU USE GOOGLE COLAB. 
# Otherwise you can COMMENT IT OUT and set up the repository LOCALLY (see instructions in README)

!mkdir helper_functions/
!mkdir images/
!curl https://raw.githubusercontent.com/mkmbader/pydata_workshop_September2024/master/requirements.txt > requirements.txt
!curl https://raw.githubusercontent.com/mkmbader/pydata_workshop_September2024/master/helper_functions/helper_functions.py > helper_functions/helper_functions.py
!curl https://raw.githubusercontent.com/mkmbader/pydata_workshop_September2024/master/helper_functions/keys.py > helper_functions/keys.py
!curl https://raw.githubusercontent.com/mkmbader/pydata_workshop_September2024/master/helper_functions/tools.py > helper_functions/tools.py

!pip install -r requirements.txt

🌟 Nice job, you finished the setup! Now you can deepdive and learn about Agents. Enjoy!

**Contents:**

1. [Default LangChain tools](#1)
        <ol type="a">
        <li>[Exercise 1 (a): Explore tool parameters](#2)</li>
        <li>[Exercise 1 (b): Run tool and explore output](#3)</li>
        </ol>
2. [Custom tools](#4)
        <ol type="a">
        <li>[Exercise 2 (a): Build your own Weather tool](#5)</li>
        <li>[Exercise 2 (b): Build your own Image tool](#6)</li>
        </ol>

3. [What are Agents](#7)
        <ol type="a">
        <li>[Exercise 1: Explore the agent's output](#8)</li>
        <li>[Exercise 2 : Build your own agent](#9)</li>
        <li>[Exercise 3: Invoke as many tools as you can](#10)</li>
        <li>[Exercise 4 : Optimize the agent prompt](#11)</li>
        </ol>


## 🛠️ Part 1: Tools
---

**The goal:** 

With this part of the notebook you will familiarize yourself with the key concepts of Tools as building blocks of an LLM Agent. At the end, you will have all the code you need to use custom LangChain tools as well as build your own custom tools.

🌟 So ... let us begin!  

**Reminder:** Make sure to update `helper_functions/keys.py` based on keys in [privatebin](https://privatebin.molops.io/?6ceb2f4c8eabe1d9#HRPAYHTvPraUrdjzU4sHaF6rYA9Snhs23bQxjj2N2cZy)

In [4]:
import io
import requests
from PIL import Image


# Update helper_functions.keys.py based on private bin link
from helper_functions.keys import WEATHER_KEY, HUGGING_FACE_KEY, OPENAI_KEY

from langchain import hub
from langchain.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import AgentExecutor # execute agent
from langchain_openai import ChatOpenAI # call openAI as agent llm
from langchain.agents import create_tool_calling_agent # set up the agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


---

### Default Langchain Tools
#### Default [Wikipedia tool](https://python.langchain.com/v0.1/docs/integrations/tools/wikipedia/) <a id='1'></a>

The cell below loads the full wikipedia tool. It makes an API call to Wikipedia using the ``WikipediaAPIWrapper`` and returns a summary of the queried article. ``WikipediaQueryRun`` then wraps this into a ready made tool. 

Each tool is a ``BaseTool`` class object, you can find its definition [here](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool).

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

#### Exercise 1.1 (a): Explore tool parameters <a id='2'></a> [optional]

**TASK:**  
Use the methods ``name``, ``description``, ``args``, ``return_direct``, ``metadata`` to familiarize yourself with the parameters of the tool. What is the meaning of the different parameters?

In [ ]:
print("Name: ", wiki_tool.name)

# TODO: insert your code here

#### Exercise 1.1 (b): Run tool and explore output <a id='3'></a> [optional]

**TASK:**
* Use the ``.run(tool_input)`` method to execute the tool. The ``tool_input`` is the search term that you'd like to query wikipedia with.
* [Optional] Check out the arguments of the WikipediaAPIWrapper [here](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html) and modify its parameters above. How does the output change? 

In [ ]:
tool_input = """
TODO: insert your code here
"""

# Run tool
# TODO: insert your code here

---

### Custom tools <a id='4'></a>
#### Custom Wikipedia tool

You can build your own tools and don't have to rely on default tools. Tools can be built from any function with the LangChain class method ``StructuredTool.from_function()``(see [here](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/#structuredtool-dataclass)). The basic elements are:
* The **function** you would like to be executed when the tool is called
* The definition of the **input parameters**
* The tool **description**

The tool description is especially important, since this is what the agent will use to make the decision if this tool should be used.

Below you see the wikipedia tool, built from the basic elements described above:

In [ ]:
# define the function
def wikipedia_caller(query:str) ->str:
    """This function queries wikipedia through a search query."""
    return api_wrapper.run(query)

# Input parameter definition
class QueryInput(BaseModel):
    query: str = Field(description="Input search query")

# the tool description
description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query."
    )


# fuse the function, input parameters and description into a tool. 
my_own_wiki_tool = StructuredTool.from_function(
    func=wikipedia_caller,
    name="wikipedia",
    description=description,
    args_schema=QueryInput,
    return_direct=False,
)

# test the output of the tool
print(my_own_wiki_tool.run('Amsterdam'))

#### Exercise 1.2 (a): Build your own Weather tool <a id='5'></a>
The goal is to build a tool that extracts weather information from the weather site visualcrossing.com. You typically need an API key to extract information from a website. In this example we provide you with the API key. 

**TASK:** 
- Build the tool by defining the input parameters and the descriptions. The tool function is already provided to you. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``.
- Test if the tool gives an output.

In [ ]:
# define the function
def extract_city_weather(city:str)->str:

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={WEATHER_KEY}&unitGroup=metric"

    response = requests.get(url)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    # insert your code here

# the tool description
description: str = (
        # TODO: insert your code here
    )

# fuse the function, input parameters and description into a tool. 
my_weather_tool = StructuredTool.from_function(
    # TODO: insert your code here
)

In [ ]:
# Test the output of your Tool
print(my_weather_tool.run('Amsterdam'))

# TODO: Try generating more ouputs

---

Let's do something even more fun. As we previously saw, we can utilize APIs to build tools. Thinking about APIs, one of the biggest collection of models are available via APIs on HuggingFace. So, how about we try to utilize this. 

#### Exercise 1.2 (b): Build your own Image tool <a id='6'></a>
The goal is to build a tool that generates an image based on a given prompt. **That means that later when you can build the Agent you can have an LLM that not only outputs text, but also images!**  

To develop this, you can make use of `Stable Diffusion v2-1`, text-to-image model available on HuggingFace. You will use the HuggingFace token that you created in the start. 

**TASK:** 
- Build the tool by defining the input parameters and the descriptions. The tool function is already provided to you. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``.
- Test if the tool gives an output.

In [ ]:
def text_to_image(payload:str):

    # Call the text-to-image API with the provided palaod
    API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-2-1"
    headers = {"Authorization": f"Bearer {HUGGING_FACE_KEY}"}

    def query(payload):
        response = requests.post(API_URL, headers=headers, json=payload)
        return response.content
    
    image_bytes = query({
        "inputs": payload,
    })

    image = Image.open(io.BytesIO(image_bytes))
    
    # Resize the image
    new_size = (400, 400)  # Example new size (width, height)
    resized_image = image.resize(new_size)


    # Save the resized image to a file
    image_path = f'images/image_{payload.replace(" ", "_")}.jpg'
    resized_image.save(image_path)
    
    # Return the path to the saved image
    return f'{image_path} '


# Input parameter definition
class ImageInput(BaseModel):
    payload: str = Field(description=
        # TODO: insert your code here
    )


# the tool description
images_description: str = (
       # TODO: insert your code here
    )

# fuse the function, input parameters and description into a tool. 
my_image_tool = StructuredTool.from_function(
        # TODO: insert your code here
)

In [ ]:
# Test the output of your Tool
print(my_image_tool.run('Amsterdam'))

# TODO: Try generating more ouputs

---

**If you developed an additional tool, make sure to copy your code in `helper_functions/tools.py` in order to later be able to use your tool in an Agent.**

Your collection of tools is now ready to be used by an agent.

🌟 Done - you are now ready to build an Agent!

---

## 🤖 Part 2: Agents
---

**The goal:** 

With this part of the notebook you will familiarize yourself with the key concepts of an LLM agent. At the end, you will have all the code you need for your very own agent that uses the tools that you developed in the previous notebook.

🌟 So ... let us begin!  

---

### What are Agents <a id='7'></a>

Agents combine the functionality of two components: LLMs and Tools. They empower an LLM to be able to execute additional tasks and reason through a problem. Namely, LLMs have knowlegde on data that was used at time of training. However, they lack knowledge about up-to-date happenings and information. They consist of the following components: 
- **LLM**: A pre-trained LLM.
- **List of tools**: List of tools that give additional functionality to the LLM.

One use case of LLM Agents is to make it possible to have LLMs with access to real time information, like the current weather. Namely, when a prompt is called, agents have an LLM and Tools at their disposal. If no tool can be found to help in answering the question, the agent tries to answer using the raw LLM. E.g. for a given prompt "What is the **usual** temperature in Amsterdam in winter?", an LLM will likely already have knowledge. However, for a prompt "What is the **current** temperature in Amsterdam?", a weather API would be a better source of information, and in this case the Agent will decide to use the information from a weather tool. If such a tool is not available to the agent, the agent will respond that the requested information is not available. 

So, basically, you can think of Agents as usual LLMs but with more "skills". Cool, right?

Let's see this through an example. 

In [ ]:
# Run this if you didn't manage to complete all Tools exercises. 
# You can use the pre-built solutions.
# Otherwise comment it out.

from helper_functions.tools import * 

In [ ]:
# Load Tools
tools = [my_own_wiki_tool, weather_tool, image_tool]

# Load LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=OPENAI_KEY)

# With this you let the agent know what its purpose is.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a nice assistant"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Define the agent (load the LLM and the list of tools)
agent = create_tool_calling_agent(llm = llm, tools = tools, prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Your agent is ready.")


---

#### Exercise 2.1: Explore the agent's output <a id='8'></a>

**TASK:**
In the examples below, read the output to observe how the Agent reasons and decides to use a different tool based on the context.

In [ ]:
question_1 = "Where is Amsterdam?"


print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

In [ ]:
question_2 = "What is the current temperature in Amsterdam?"

print(f"Question 1: {question_2}")
agent_executor.invoke({"input": question_2})

In [ ]:
question_3 = "What should I visit in Amsterdam? Show me a photo"

print(f"Question 1: {question_3}")
agent_executor.invoke({"input": question_3})

---

#### Exercise 2.2 : Build your own agent  <a id='9'></a>
The goal is to build an agent that uses the tools you previously developed. Feel free to also use the pre-made tools, available at `helper_functions/tools.py`

**TASK**: 
- A template for defining an agent and an API key are already provided to you. Build an agent by using a list of tools, and the LLM `gpt-4o-mini`.
- Test if the agent gives an output.
- Observe how the output changes if you provide less tools in your list of tools.
- Observe how the output changes if you change the [temperature](https://www.iguazio.com/glossary/llm-temperature/) of the LLM.

In [ ]:
# Component 1 (Tools): Load Tools
tools = [
# TODO: insert your code here
]

# Component 2 (LLM): Load LLM
llm = ChatOpenAI(model= # TODO: insert your code here           
                 temperature=0, 
                 api_key=OPENAI_KEY)

# Component 3 (Prompt): Let the agent know what its purpose is. For now, let's keep it as is.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a nice assistant"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Define the agent and agent executor (load the LLM, the list of tools, and the prompt (descripiton))
agent = create_tool_calling_agent(
    # TODO: insert your code here
    )
agent_executor = AgentExecutor(
    # TODO: insert your code here
)

print("Your agent is ready.")


---

#### Exercise 2.3: Invoke as many tools as you can  <a id='10'></a>

**TASK:**
Try various questions to call the agent and follow the generated reasoning process in the response. The goal is to call the agent in a way thay it will use as many tools as possible. **Let's see who can reach the highest number of tools used with a single prompt!**

In [ ]:
question = """
#TODO: Replace this with your question 
"""

print(f"Question: {question}")
agent_executor.invoke({"input": question})

---

#### Exercise 2.4 : Optimize the agent prompt <a id='11'></a>
So far we played around with the provided LLM and list of tools. Now let's look into the 3rd component: the Agent prompt. 

**TASK:**
- Modify the agent prompt and observe the difference in the output. 

In [ ]:
# Component 1 (Tools): Load Tools from Exercise 1
tools = tools 

# Component 2 (LLM): Load LLM from Exercise 1
llm = llm

# Component 3 (Prompt): Create your own prompt to instruct the Agent about its purpose.
your_prompt = """"
    #  TODO: enter your code here
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", your_prompt),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt.messages


# This is same as in Exercise 1
# Define the agent and agent executor (load the LLM, the list of tools, and the prompt (descripiton))
agent = create_tool_calling_agent(
    llm = llm, tools = tools, prompt = prompt
    )
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

print("Your agent is ready.")


In [ ]:
# Observe how the same question from before is answered differently with the different prompt.
print(f"Question: {question}")
agent_executor.invoke({"input": question})

---

🌟 Good job! - You are now ready to proceed to the next notebook.